In [1]:
import sys
sys.path.insert(1, 'bin')

from persistence import sql
import functional as fn
import color as c

# Print all commands

In [ ]:
db = sql('functional')

for i in db.query('SELECT * FROM command'):
    print(c.blue + i['class'] + ':' + i['name'])
    print(c.white + fn.build(i['class'], i['name']))

db.close()

# Create command

if a command adds a new parameter it must be specified

In [ ]:
fn.create_command('python','if','''if({{condition}}):
  {{block}}''',['condition', 'block'])

# Create command and add child commands

(this command does not have valid sintax)

In [15]:
fn.create_command('debug', 'test')
fn.set_child(('debug', 'test'),('python','if'),{'block':('select','from')})

# Example

In [3]:
print(fn.build('python','if',{'condition':'true','block':'print("hello world")\nprint("line2")'}))

if(true):
  print("hello world")
  print("line2")


# Example2

In [20]:
def cat(a, b, separator = ' '):
    return [a[i] + separator + b[i] for i in range(len(a) if len(a) < len(b) else len(b))]

class table:
    def __init__(self, table, field):
        self.table = table
        self.field = []
        self.tipo = []

        for i in field:
            self.field.append(i)
            self.tipo.append(field[i])
            
        self.value = []
        for i in field:
            self.value.append('@' + i.capitalize()) 
        
        self.id = 'id' == self.field[0]

    @property
    def create(self):
        field = cat(self.field, self.tipo)
        if(self.id):
            field.append('PRIMARY KEY (id)')
        return fn.build('sql', 'create', [
            self.table,
            field
        ]) + ';\n'
    
    @property
    def insert(self):
        return fn.build('sql', 'insert values', [
            self.table,
            self.field[1::] if self.id else self.field,
            self.value[1::] if self.id else self.value
        ]) + ';\n'
    
    @property
    def select(self):
        return fn.build('sql', 'select from', [
            self.field,
            self.table
        ]) + ';\n'
    
    @property
    def update(self, condition = None):
        if(not self.id and not condition):
            return c.red + 'condition required' + c.white
        return fn.build('sql', 'update set where', [
            self.table,
            cat
            (
                self.field[1::] if self.id else self.field,
                self.value[1::] if self.id else self.field,
                ' = '
            ),
            cat
            (
                self.field[:1],
                self.value[:1],
                ' = '
            ) if self.id else condition
        ])+ ';\n'


pessoa = table(
    'pessoa',
{
    'id': 'INT AUTO_INCREMENT',
    'nome': 'VARCHAR(32)',
    'sexo': 'CHAR(1)',
    'idade': 'INT'
})

print(pessoa.create)
print(pessoa.insert)
print(pessoa.select)
print(pessoa.update)



CREATE TABLE 
  pessoa
(
  id INT AUTO_INCREMENT,
  nome VARCHAR(32),
  sexo CHAR(1),
  idade INT,
  PRIMARY KEY (id)
);

INSERT INTO
  pessoa 
(
  nome,
  sexo,
  idade
)
VALUES
(
  @Nome,
  @Sexo,
  @Idade
);

SELECT
  id,
  nome,
  sexo,
  idade
FROM
  pessoa;

UPDATE
  pessoa
SET
 nome = @Nome,
 sexo = @Sexo,
 idade = @Idade
WHERE
  id = @Id;

